In [11]:
import warnings

import astropy.units
import FunctionLib as FL
import inspect
from tqdm import tqdm
import astropy
import wave
from matplotlib.image import resample
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
from collections import defaultdict
import re
import scipy

mpl.rcParams['font.family'] = 'serif'


warnings.filterwarnings("ignore")

DJAv4Catalog = FL.Spectrum_Catalog()
DJAv4Catalog.load_from_pkl(os.path.expanduser(
    '~/DustCurve/DJAv4Catalog.pkl'))
print(DJAv4Catalog.sample_num())

DJAv4Catalog.to_dataframe()


1170


,survey_id_subid,survey_id,prism_filepath,prism_redshift,grating_filepaths,grating_redshifts,determined_redshift,file_count,available_filters,properties
0,snh0pe-v4_4446_102,snh0pe-v4,/home/xingyaocai/DJAv4/snh0pe-v4/snh0pe-v4_pri...,0.2259,{'g140m-f100lp': '/home/xingyaocai/DJAv4/snh0p...,"{'g140m-f100lp': nan, 'g235m-f170lp': nan}",0.2259,3,"{g140m-f100lp, prism-clear, g235m-f170lp}","{'redshift_conflict': False, 'Sample_Flag': Fa..."
1,snh0pe-v4_4446_143,snh0pe-v4,/home/xingyaocai/DJAv4/snh0pe-v4/snh0pe-v4_pri...,1.6318,{'g140m-f100lp': '/home/xingyaocai/DJAv4/snh0p...,"{'g140m-f100lp': 1.6313, 'g235m-f170lp': 1.6309}",1.6311,3,"{g140m-f100lp, prism-clear, g235m-f170lp}","{'redshift_conflict': False, 'Sample_Flag': Fa..."
2,snh0pe-v4_4446_285,snh0pe-v4,/home/xingyaocai/DJAv4/snh0pe-v4/snh0pe-v4_pri...,0.4446,{'g140m-f100lp': '/home/xingyaocai/DJAv4/snh0p...,"{'g140m-f100lp': 0.4462, 'g235m-f170lp': 0.4462}",0.4462,3,"{g140m-f100lp, prism-clear, g235m-f170lp}","{'redshift_conflict': False, 'Sample_Flag': Fa..."
3,snh0pe-v4_4446_29,snh0pe-v4,/home/xingyaocai/DJAv4/snh0pe-v4/snh0pe-v4_pri...,1.7834,{'g140m-f100lp': '/home/xingyaocai/DJAv4/snh0p...,"{'g140m-f100lp': 1.7799, 'g235m-f170lp': 1.7796}",1.77975,3,"{g140m-f100lp, prism-clear, g235m-f170lp}","{'redshift_conflict': False, 'Sample_Flag': Fa..."
4,snh0pe-v4_4446_123,snh0pe-v4,/home/xingyaocai/DJAv4/snh0pe-v4/snh0pe-v4_pri...,1.7855,{'g140m-f100lp': '/home/xingyaocai/DJAv4/snh0p...,"{'g140m-f100lp': 1.7855, 'g235m-f170lp': 1.7851}",1.7855,3,"{g140m-f100lp, prism-clear, g235m-f170lp}","{'redshift_conflict': False, 'Sample_Flag': Fa..."
...,...,...,...,...,...,...,...,...,...,...
32564,ceers-ddt-v4_2750_1636,ceers-ddt-v4,/home/xingyaocai/DJAv4/ceers-ddt-v4/ceers-ddt-...,3.0592,{},{},3.0592,1,{prism-clear},"{'redshift_conflict': False, 'Sample_Flag': Fa..."
32565,ceers-ddt-v4_2750_3213,ceers-ddt-v4,/home/xingyaocai/DJAv4/ceers-ddt-v4/ceers-ddt-...,4.8013,{},{},4.8013,1,{prism-clear},"{'redshift_conflict': False, 'Sample_Flag': Fa..."
32566,ceers-ddt-v4_2750_340,ceers-ddt-v4,/home/xingyaocai/DJAv4/ceers-ddt-v4/ceers-ddt-...,0.9585,{},{},0.9585,1,{prism-clear},"{'redshift_conflict': False, 'Sample_Flag': Fa..."
32567,ceers-ddt-v4_2750_21413,ceers-ddt-v4,/home/xingyaocai/DJAv4/ceers-ddt-v4/ceers-ddt-...,2.1402,{},{},2.1402,1,{prism-clear},"{'redshift_conflict': False, 'Sample_Flag': Fa..."


In [12]:
def Good_Fit_Results(fit_result, line_wavelength):
    if fit_result is None:
        return False
    if fit_result['success'] is False:
        return False
    if fit_result['parameters']['amplitude'].value <= 1e-20:
        return False
    if fit_result['parameters']['mean'] < line_wavelength - 10 * astropy.units.AA or fit_result['parameters']['mean'] > line_wavelength + 10 * astropy.units.AA:
        return False
    if fit_result['parameters']['stddev'] >= 6 * astropy.units.AA:
        return False
    if fit_result['parameters']['stddev'] <= 0.1 * astropy.units.AA:
        return False
    return True

In [19]:
alphacount=0
betacount=0
count=0
no_line_count=0
for id, catalog in tqdm(DJAv4Catalog.catalog_iterator(), desc='Processing DJAv4Catalog'):
    if not catalog['properties']['Sample_Flag']:
        continue

    Halpha_Flag=False
    Hbeta_Flag=False
    Halpha_Fit_Result=None
    Hbeta_Fit_Result=None


    for filter, grating_path in catalog['grating_filepaths'].items():
        grating_spectrum=FL.Load_Spectrum_From_Fits(grating_path,catalog['determined_redshift'])
        inf,sup= grating_spectrum.dual_boundarys()
        if inf< 6563.0*astropy.units.AA and sup>6563.0*astropy.units.AA:
            Halpha_Flag=True
        if inf< 4863.0*astropy.units.AA and sup>4863.0*astropy.units.AA:
            Hbeta_Flag=True

        if Halpha_Flag:
            grating_spectrum.set_boundarys(6563.0*astropy.units.AA-100*astropy.units.AA, 6563.0*astropy.units.AA+100*astropy.units.AA)
            if len(grating_spectrum.processing_flux.data) < 10:
                continue
            Halpha_Fitter=FL.SpectralLineFitter(grating_spectrum,6563.0*astropy.units.AA)


            initial_amplitude_guess=(grating_spectrum.processing_flux[np.argmin(np.abs(grating_spectrum.processing_wavelengths-6563.0*astropy.units.AA))]).data

            initial_mean_guess=grating_spectrum.processing_wavelengths[np.argmin(np.abs(grating_spectrum.processing_wavelengths-6563.0*astropy.units.AA))].data

            initial_sigma_guess=2.0

            initial_offset_guess=np.median(grating_spectrum.processing_flux.data)


            initial_guess=[initial_amplitude_guess, initial_mean_guess, initial_sigma_guess,initial_offset_guess]

            Halpha_Fit_Result=Halpha_Fitter.fit_single_gaussian_with_offset(initial_guess)

        if Hbeta_Flag:
            grating_spectrum.set_boundarys(4863.0*astropy.units.AA-100*astropy.units.AA, 4863.0*astropy.units.AA+100*astropy.units.AA)
            if len(grating_spectrum.processing_flux.data) < 10:
                continue
            Hbeta_Fitter=FL.SpectralLineFitter(grating_spectrum,4863.0*astropy.units.AA)


            initial_amplitude_guess=(grating_spectrum.processing_flux[np.argmin(np.abs(grating_spectrum.processing_wavelengths-4863.0*astropy.units.AA))]).data

            initial_mean_guess=grating_spectrum.processing_wavelengths[np.argmin(np.abs(grating_spectrum.processing_wavelengths-4863.0*astropy.units.AA))].data

            initial_sigma_guess=2.0

            initial_offset_guess=np.median(grating_spectrum.processing_flux.data)


            initial_guess=[initial_amplitude_guess, initial_mean_guess, initial_sigma_guess,initial_offset_guess]

            Hbeta_Fit_Result=Hbeta_Fitter.fit_single_gaussian_with_offset(initial_guess)

    if Halpha_Fit_Result is None or Hbeta_Fit_Result is None:
            continue

    if Halpha_Fit_Result is not None:

        if Good_Fit_Results(Halpha_Fit_Result, 6563.0 * astropy.units.AA):
            alphacount += 1
            catalog['properties']['Halpha_Fit_Result'] = Halpha_Fit_Result
            catalog['properties']['Halpha_Flag'] = True
    if Hbeta_Fit_Result is not None:

        if Good_Fit_Results(Hbeta_Fit_Result, 4863.0 * astropy.units.AA):
            betacount += 1
            catalog['properties']['Hbeta_Fit_Result'] = Hbeta_Fit_Result
            catalog['properties']['Hbeta_Flag'] = True

    if Good_Fit_Results(Halpha_Fit_Result,6563.0*astropy.units.AA) and Good_Fit_Results(Hbeta_Fit_Result,4863.0*astropy.units.AA):
        count+=1
        catalog['properties']['Both_Lines_Fit_Flag'] = True
    else:
        catalog['properties']['Both_Lines_Fit_Flag'] = False
    if not Halpha_Fit_Result and not Hbeta_Fit_Result:
        no_line_count+=1
    DJAv4Catalog.update_catalog_item(id, catalog)


Processing DJAv4Catalog: 32569it [00:32, 994.80it/s] 


In [20]:
print(f"Number of galaxies with good H-alpha fits: {alphacount}")
print(f"Number of galaxies with good H-beta fits: {betacount}")
print(f"Number of galaxies with good H-alpha and H-beta fits: {count}")


Number of galaxies with good H-alpha fits: 600
Number of galaxies with good H-beta fits: 839
Number of galaxies with good H-alpha and H-beta fits: 459


In [ ]:
alphacount=0
betacount=0
count=0
for id, catalog in DJAv4Catalog.catalog_iterator():
    if not catalog['properties']['Sample_Flag']:
        continue

    if 'Halpha_Fit_Result' in catalog['properties'].keys():
        if Good_Fit_Results(catalog['properties']['Halpha_Fit_Result'], 6563.0 * astropy.units.AA):
            alphacount += 1
    if 'Hbeta_Fit_Result' in catalog['properties'].keys():
        if Good_Fit_Results(catalog['properties']['Hbeta_Fit_Result'], 4863.0 * astropy.units.AA):
            betacount += 1
    if 'Both_Lines_Fit_Flag' in catalog['properties'].keys():
        if catalog['properties']['Both_Lines_Fit_Flag']:
            count += 1
        else:
            catalog['properties']['Sample_Flag'] = False
            catalog['properties']['Sample_Reason'] = 'No good fit for both H-alpha and H-beta lines'
    else:
        catalog['properties']['Sample_Flag'] = False
        catalog['properties']['Sample_Reason'] = 'No good fit for both H-alpha and H-beta lines'
    DJAv4Catalog.update_catalog_item(id, catalog)

print(f"Number of galaxies with good H-alpha fits: {alphacount}")
print(f"Number of galaxies with good H-beta fits: {betacount}")
print(f"Number of galaxies with good H-alpha and H-beta fits: {count}")


Number of galaxies with good H-alpha fits: 600
Number of galaxies with good H-beta fits: 839
Number of galaxies with good H-alpha and H-beta fits: 459


In [22]:
DJAv4Catalog.sample_num()

459